#Big Data Management
Databricks + Spark  
Garth Mortensen  
2020.03.06

Render markdown with [%md](https://forums.databricks.com/answers/9506/view.html)

## Tasks

2. Filter out header rows
3. Provide an output that has employee count by department ID
4. Provide an output that has the top two department names with the most employee ID.

## Execution

### Read files as RDDs

We want to read the files in as RDDs. 

First, I upload the two files to Databricks cluster, then use shell to see if the files are in the right place.

In [2]:
%sh
cd /dbfs/FileStore/tables
ls -la

total 0
drwxr-xr-x 1 root root 0 Jan 1 1970 .
drwxr-xr-x 1 root root 0 Jan 1 1970 ..
-rw-r--r-- 1 root root 12859382 Mar 22 14:33 CAND_MSI_UNI_JS.json
-rw-r--r-- 1 root root 5139 Mar 7 02:55 cities.txt
-rw-r--r-- 1 root root 18513237 Mar 22 14:32 CMTE_MST_UNI_JS.json
-rw-r--r-- 1 root root 60930417 Mar 22 14:34 CONTRIB_TO_CMTE_FR_CMTE_MN_JS.json
-rw-r--r-- 1 root root 107 Mar 21 17:51 Dept1.csv
-rw-r--r-- 1 root root 107 Mar 8 14:43 Dept.csv
-rw-r--r-- 1 root root 714 Mar 21 17:51 Emp.csv
-rw-r--r-- 1 root root 243 Mar 21 17:51 Emp.json
-rw-r--r-- 1 root root 664 Mar 8 14:43 Employee.csv

Because I have read permission, I don't need to change permissions.

```sh
%sh
cd /dbfs/FileStore/tables
ls -la
```

Good. The files are ready. Now time to figure out how to read them. Of note, I'm importing these without headers.

### Read File

#### Pandas DataFrame

In [5]:
# the pandas solution
import pandas as pd

dept_pd_df = pd.read_csv("/dbfs/FileStore/tables/Dept.csv", header=0)

emp_pd_df = pd.read_csv("/dbfs/FileStore/tables/Employee.csv", header=0)
emp_pd_df[0:5]  # print first 5 rows

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7369,SMITH,CLERK,7902.0,6/13/1993,800,0.0,20
1,7499,ALLEN,SALESMAN,7698.0,8/15/1998,1600,300.0,30
2,7521,WARD,SALESMAN,7698.0,3/26/1996,1250,500.0,30
3,7566,JONES,MANAGER,7839.0,10/31/1995,2975,NaN,20
4,7698,BLAKE,MANAGER,7839.0,6/11/1992,2850,NaN,30


### Spark DataFrame

In [7]:
# the spark solution

# Define path and read csv without headers
file_emp = "/FileStore/tables/Employee.csv"
emp_df = spark.read.csv(file_emp, header="true", inferSchema="true")

file_dept = "/FileStore/tables/Dept.csv"
dept_df = spark.read.csv(file_dept, header="true", inferSchema="true")
dept_df.take(3)

Out[7]: [Row(deptno=10, dname='Accounting', location='New York'),
 Row(deptno=20, dname='Research', location='Dallas'),
 Row(deptno=30, dname='Sales', location='Chicago')]

So I can import as pandas df or as spark df. Why would I choose one over the other? 

I'm guessing that because pandas doesn't distribute info out and in to the cluster. I bet only spark does that.

### Spark Textfile

In [10]:
employee_txt = sc.textFile("/FileStore/tables/Employee.csv")
employee_txt.take(5)

Out[66]: ['empno,ename,job,mgr,hiredate,sal,comm,deptno',
 '7369,SMITH,CLERK,7902,6/13/1993,800,0,20',
 '7499,ALLEN,SALESMAN,7698,8/15/1998,1600,300,30',
 '7521,WARD,SALESMAN,7698,3/26/1996,1250,500,30',
 '7566,JONES,MANAGER,7839,10/31/1995,2975,,20']

In [11]:
dept_txt = sc.textFile("/FileStore/tables/Dept.csv")
dept_txt.take(5)

Out[67]: ['deptno,dname,location',
 '10,Accounting,New York',
 '20,Research,Dallas',
 '30,Sales,Chicago',
 '40,Operations,Boston']

#### Spark Remove Headers

In [13]:
header = employee_txt.first()  # pull first row
employee_txt = employee_txt.filter(lambda row: row != header)
employee_txt.take(5)

Out[68]: ['7369,SMITH,CLERK,7902,6/13/1993,800,0,20',
 '7499,ALLEN,SALESMAN,7698,8/15/1998,1600,300,30',
 '7521,WARD,SALESMAN,7698,3/26/1996,1250,500,30',
 '7566,JONES,MANAGER,7839,10/31/1995,2975,,20',
 '7698,BLAKE,MANAGER,7839,6/11/1992,2850,,30']

In [14]:
header = dept_txt.first()
dept_txt = dept_txt.filter(lambda row: row != header)
dept_txt.take(5)

Out[69]: ['10,Accounting,New York',
 '20,Research,Dallas',
 '30,Sales,Chicago',
 '40,Operations,Boston']

### Group By Setup

Find employee count by department ID. 

This could be either a:
1. [RDD Action](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions), which runs a computation on the dataset, then returns the value to the driver program.
2. [RDD Transformation](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations), which creates a new dataset from an existing one.

reduceByKey (func, [numPartitions]) - When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V,V) => V. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.

In [16]:
%sql
-- the sql solution
-- select count(*)
-- from emp_df
-- group by deptno

#### Pandas DataFrame

In [18]:
# the pandas solution
emp_pd_df.groupby("deptno")["empno"].count()

Out[8]: deptno
10 3
20 5
30 6
Name: empno, dtype: int64

Let's first preview our data.

#### Spark DataFrame

In [21]:
# the spark solution
emp_df.printSchema()

root
-- empno: integer (nullable = true)
-- ename: string (nullable = true)
-- job: string (nullable = true)
-- mgr: integer (nullable = true)
-- hiredate: string (nullable = true)
-- sal: integer (nullable = true)
-- comm: integer (nullable = true)
-- deptno: integer (nullable = true)

In [22]:
emp_df.show()

+-----+------+---------+----+----------+----+----+------+
empno| ename| job| mgr| hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
 7369| SMITH| CLERK|7902| 6/13/1993| 800| 0| 20|
 7499| ALLEN| SALESMAN|7698| 8/15/1998|1600| 300| 30|
 7521| WARD| SALESMAN|7698| 3/26/1996|1250| 500| 30|
 7566| JONES| MANAGER|7839|10/31/1995|2975|null| 20|
 7698| BLAKE| MANAGER|7839| 6/11/1992|2850|null| 30|
 7782| CLARK| MANAGER|7839| 5/14/1993|2450|null| 10|
 7788| SCOTT| ANALYST|7566| 3/5/1996|3000|null| 20|
 7839| KING|PRESIDENT|null| 6/9/1990|5000| 0| 10|
 7844|TURNER| SALESMAN|7698| 6/4/1995|1500| 0| 30|
 7876| ADAMS| CLERK|7788| 6/4/1999|1100|null| 20|
 7900| JAMES| CLERK|7698| 6/23/2000| 950|null| 30|
 7934|MILLER| CLERK|7782| 1/21/2000|1300|null| 10|
 7902| FORD| ANALYST|7566| 12/5/1997|3000|null| 20|
 7654|MARTIN| SALESMAN|7698| 12/5/1998|1250|1400| 30|
+-----+------+---------+----+----------+----+----+------+

In [23]:
emp_df.describe().show()

+-------+------------------+-----+--------+------------------+---------+------------------+-----------------+------------------+
summary| empno|ename| job| mgr| hiredate| sal| comm| deptno|
+-------+------------------+-----+--------+------------------+---------+------------------+-----------------+------------------+
 count| 14| 14| 14| 13| 14| 14| 6| 14|
 mean| 7726.571428571428| null| null| 7739.307692307692| null| 2073.214285714286|366.6666666666667|22.142857142857142|
 stddev|178.29436087795662| null| null|103.71466033898376| null|1182.5032235162716|546.5040408511785| 8.017837257372731|
 min| 7369|ADAMS| ANALYST| 7566|1/21/2000| 800| 0| 10|
 max| 7934| WARD|SALESMAN| 7902|8/15/1998| 5000| 1400| 30|
+-------+------------------+-----+--------+------------------+---------+------------------+-----------------+------------------+

Now, I'll perform the Group By, and display the **top two** department names with the most employee IDs.

#### Pandas DataFrame

Do it in pandas.

In [25]:
# the pandas solution
joined = pd.merge(emp_pd_df, dept_pd_df, on='deptno', sort='deptno')
joined

,empno,ename,job,mgr,hiredate,sal,comm,deptno,dname,location
0,7782,CLARK,MANAGER,7839.0,5/14/1993,2450,NaN,10,Accounting,New York
1,7839,KING,PRESIDENT,NaN,6/9/1990,5000,0.0,10,Accounting,New York
2,7934,MILLER,CLERK,7782.0,1/21/2000,1300,NaN,10,Accounting,New York
3,7369,SMITH,CLERK,7902.0,6/13/1993,800,0.0,20,Research,Dallas
4,7566,JONES,MANAGER,7839.0,10/31/1995,2975,NaN,20,Research,Dallas
5,7788,SCOTT,ANALYST,7566.0,3/5/1996,3000,NaN,20,Research,Dallas
6,7876,ADAMS,CLERK,7788.0,6/4/1999,1100,NaN,20,Research,Dallas
7,7902,FORD,ANALYST,7566.0,12/5/1997,3000,NaN,20,Research,Dallas
8,7499,ALLEN,SALESMAN,7698.0,8/15/1998,1600,300.0,30,Sales,Chicago
9,7521,WARD,SALESMAN,7698.0,3/26/1996,1250,500.0,30,Sales,Chicago


In [26]:
joined2 = joined.groupby("dname")["empno"].count().to_frame()  # group by, and convert output series back to df
joined2.sort_values('empno', ascending=False)  # it's not perfect, but good enough. I want to see Sales and Research at the end

,empno
dname,
Sales,6
Research,5
Accounting,3


#### Spark DataFrame

In [28]:
emp_df.groupBy("deptno").count().show()

+------+-----+
deptno|count|
+------+-----+
 20| 5|
 10| 3|
 30| 6|
+------+-----+

This doesn't capture the department name. I'll need to join for that. No fancy footwork needed on the join. An inner will do the job, since there won't be anyone in the employees table without a department, we don't care about left joining.

In [30]:
df_joined = emp_df.join(dept_df, on='deptno', how='inner')  # inner is default.
df_joined.show()

+------+-----+------+---------+----+----------+----+----+----------+--------+
deptno|empno| ename| job| mgr| hiredate| sal|comm| dname|location|
+------+-----+------+---------+----+----------+----+----+----------+--------+
 20| 7369| SMITH| CLERK|7902| 6/13/1993| 800| 0| Research| Dallas|
 30| 7499| ALLEN| SALESMAN|7698| 8/15/1998|1600| 300| Sales| Chicago|
 30| 7521| WARD| SALESMAN|7698| 3/26/1996|1250| 500| Sales| Chicago|
 20| 7566| JONES| MANAGER|7839|10/31/1995|2975|null| Research| Dallas|
 30| 7698| BLAKE| MANAGER|7839| 6/11/1992|2850|null| Sales| Chicago|
 10| 7782| CLARK| MANAGER|7839| 5/14/1993|2450|null|Accounting|New York|
 20| 7788| SCOTT| ANALYST|7566| 3/5/1996|3000|null| Research| Dallas|
 10| 7839| KING|PRESIDENT|null| 6/9/1990|5000| 0|Accounting|New York|
 30| 7844|TURNER| SALESMAN|7698| 6/4/1995|1500| 0| Sales| Chicago|
 20| 7876| ADAMS| CLERK|7788| 6/4/1999|1100|null| Research| Dallas|
 30| 7900| JAMES| CLERK|7698| 6/23/2000| 950|null| Sales| Chicago|
 10| 7934|MILLER| CLERK|7782| 1/21/2000|1300|null|Accounting|New York|
 20| 7902| FORD| ANALYST|7566| 12/5/1997|3000|null| Research| Dallas|
 30| 7654|MARTIN| SALESMAN|7698| 12/5/1998|1250|1400| Sales| Chicago|
+------+-----+------+---------+----+----------+----+----+----------+--------+

In [31]:
df_joined2 = df_joined.groupBy("dname").count()
df_joined2.show()  # .show() on a variable assignment results in Type: None. Need to break it into 2 parts.

+----------+-----+
 dname|count|
+----------+-----+
 Sales| 6|
Accounting| 3|
 Research| 5|
+----------+-----+

In [32]:
df_joined2.orderBy('count', ascending=False).show(2)  # show only top 2 records

# alternative
# df_joined2.sort('count').show()

+--------+-----+
 dname|count|
+--------+-----+
 Sales| 6|
Research| 5|
+--------+-----+
only showing top 2 rows

And one last curiosity was to check the filesystem for the temp tables I've created. After hunting around for a few minutes, I couldn't locate any.

In [34]:
%sh

cd /dbfs/
ls -la

ls: cannot access 'databricks-results': No such file or directory
total 8
drwxrwxrwx 1 root root 4096 Jan 1 1970 .
drwxr-xr-x 1 root root 4096 Mar 29 12:34 ..
-rw-r--r-- 1 root root 4610348 Mar 21 19:00 bank-full.csv
drwxr-xr-x 1 root root 0 Jan 1 1970 databricks-datasets
?????????? ? ? ? ? ? databricks-results
drwxr-xr-x 1 root root 0 Jan 1 1970 FileStore
drwxrwxrwx 2 root root 4096 Mar 29 12:34 ml
drwxr-xr-x 1 root root 0 Jan 1 1970 tmp
drwxr-xr-x 1 root root 0 Jan 1 1970 user

#### Spark RDD

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4344670797617315> in <module> 
 ----> 1 header = employee_txt . first ( ) # pull first row 
 2 employee1 = employee_txt . filter ( lambda row : row != header ) 
 3 employee1 . take ( 5 ) 

 /databricks/spark/python/pyspark/rdd.py in first (self) 
 1417 ValueError : RDD is empty
 1418 """
 -> 1419 rs = self . take ( 1 ) 
 1420 if rs : 
 1421 return rs [ 0 ] 

 /databricks/spark/python/pyspark/rdd.py in take (self, num) 
 1399 
 1400 p = range ( partsScanned , min ( partsScanned + numPartsToTry , totalParts ) ) 
 -> 1401 res = self . context . runJob ( self , takeUpToNumLeft , p ) 
 1402 
 1403 items += res

 /databricks/spark/python/pyspark/context.py in runJob (self, rdd, partitionFunc, partitions, allowLocal) 
 1176 finally : 
 1177 os . remove ( filename ) 
 -> 1178 sock_info = self . _jvm . PythonRDD . runJob ( self . _jsc . sc ( ) , mappedRDD . _jrdd , partitions ) 
 1179 return list ( _load_from_socket ( sock_info , mappedRDD . _jrdd_deserializer ) ) 
 1180 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 134.0 failed 1 times, most recent failure: Lost task 0.0 in stage 134.0 (TID 868, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 480, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 470, in process
 out_iter = func(split_index, iterator)
 File "/databricks/spark/python/pyspark/rdd.py", line 2542, in pipeline_func
 return func(split, prev_func(split, iterator))
 File "/databricks/spark/python/pyspark/rdd.py", line 2542, in pipeline_func
 return func(split, prev_func(split, iterator))
 File "/databricks/spark/python/pyspark/rdd.py", line 353, in func
 return f(iterator)
 File "/databricks/spark/python/pyspark/rdd.py", line 1904, in combineLocally
 merger.mergeValues(iterator)
 File "/databricks/spark/python/pyspark/shuffle.py", line 238, in mergeValues
 for k, v in iterator:
 File "/databricks/spark/python/pyspark/util.py", line 99, in wrapper
 return f(*args, **kwargs)
 File "<command-4344670797617313>", line 2, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:540)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:676)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:659)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:494)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1124)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1130)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(Byp

In [37]:
emp1 = employee_txt.map(lambda x: x.split(','))
emp2 = emp1.map(lambda x: (x[7],1))
emp3 = emp2.reduceByKey(lambda x, y: x + y)
emp3.take(5)  # this is not working.

Out[76]: [('20', 5), ('10', 3), ('deptno', 1), ('30', 6)]

In [38]:
join_rdd = employee_txt.leftOuterJoin(dept_txt)
join_rdd.take(5)  # no good!

Out[77]: [('e', ('m', None)),
 ('7', ('3', None)),
 ('7', ('4', None)),
 ('7', ('5', None)),
 ('7', ('5', None))]